In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import savemat
import shutil
import subprocess
import dartsort
import tempfile
import dartsort.vis as dartvis

In [7]:
scaleprocs_dir = Path("~/proj/measims/scaleproc_recordings").expanduser()
rez_dir = Path("~/proj/measims/ksrez").expanduser()
ks_dir = Path("~/Kilosort/").expanduser()
si2ks_py = ks_dir / "si2ks.py"
rez_dir.mkdir(exist_ok=True)
assert si2ks_py.exists()
list(scaleprocs_dir.iterdir())

[PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_bumps_uniform_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_static_bimodal_modulated'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_static_uniform_modulated'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_rigid_uniform_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_rigid_bimodal_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_bumps_bimodal_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_bumps_uniform_modulated'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_static_uniform_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_rigid_bimodal_modulated'),
 PosixPath('/home/cwindolf/proj/measims/scaleproc_r

In [8]:
!ls /home/cwindolf/proj/measims/scaleproc_recordings/Neuropixels-128_bumps_uniform_homogeneous

binary.json	     probe.json  provenance.json  traces_cached_seg0.raw
kilosortChanMap.mat  properties  si_folder.json


In [10]:
nblocks = 2
nBinsReg = 15
depthBin = 5
nc = 128
config_m = "/home/cwindolf/spike_sorting_motion_benchmark_paper_2023/configFileMEArec128.m"
prefix_cmd = "ml load matlab/2022b"

In [13]:
for p in scaleprocs_dir.iterdir():
    # make a nice house for KS to live in
    out_dir = rez_dir / p.name
    out_dir.mkdir(exist_ok=True)
    (out_dir / "f.dat").symlink_to(p / "traces_cached_seg0.raw")

    # run KS
    # this just runs registration in modded-v2.5
    cmpath = p / "kilosortChanMap.mat"
    with tempfile.TemporaryDirectory() as scratchdir:
        cmd = (
            f"main_kilosort('{out_dir}', "
            f"'{scratchdir}', "
            f"'{config_m}', '{cmpath}', "
            f"0, Inf, {nc}, {nBinsReg}, {depthBin}, {nblocks})"
        )

        matlab_cmd = f'matlab -nodisplay -nosplash -r "{cmd}; exit;"'
        fullcmd = f"cd {ks_dir} && {matlab_cmd}"
        if prefix_cmd:
            fullcmd = f"{prefix_cmd} && {fullcmd}"

        subprocess.run(
            fullcmd,
            shell=True,
        )


                            < M A T L A B (R) >
                  Copyright 1984-2022 The MathWorks, Inc.
             R2022b Update 2 (9.13.0.2105380) 64-bit (glnxa64)
                              October 26, 2022

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
Looking for data inside /home/cwindolf/proj/measims/ksrez/Neuropixels-128_bumps_uniform_homogeneous 
ops
ops = 

  struct with fields:

            trange: [0 Inf]
          NchanTOT: 128
           chanMap: [1x1 struct]
                fs: 32000
            fshigh: 300
                Th: [9 9]
               lam: 20
          AUCsplit: 0.8000
             minFR: 0.0200
         sigmaMask: 30
             ThPre: 8
               sig: 20
             spkTh: -6
           reorder: 1
             nskip: 25
               GPU: 1
      nfilt_factor: 4
            ntbuff: 64
                NT: 65600
    whiteningRange: 32
          nSkipCov: 25
         scaleproc: 200
              nPCs: 3
       